# Using results_df for Plan Results Summary

This notebook demonstrates the `results_df` DataFrame, which provides lightweight HDF-based results summaries for all plans. This enables quick access to execution status, volumetric errors, compute messages, and runtime data without opening heavy geospatial results.

## Setup and Imports

In [ ]:
from pathlib import Path
import sys

# Flexible imports for development vs installed package
try:
    from ras_commander import init_ras_project, RasCmdr, RasExamples, ras
    from ras_commander.results import ResultsParser
except ImportError:
    current_file = Path(__file__).resolve()
    parent_directory = current_file.parent.parent
    sys.path.append(str(parent_directory))
    from ras_commander import init_ras_project, RasCmdr, RasExamples, ras
    from ras_commander.results import ResultsParser

## Extract Example Project

In [ ]:
# Extract Muncie example project
project_path = RasExamples.extract_project("Muncie")
print(f"Project extracted to: {project_path}")

## Initialize Project with results_df

When initializing a project, `results_df` is automatically populated by default (`load_results_summary=True`).

In [ ]:
# Initialize project - results_df is automatically loaded
init_ras_project(project_path, "6.6")

print(f"Plans in project: {len(ras.plan_df)}")
print(f"Results summaries loaded: {len(ras.results_df)}")

## View Initial results_df

Before execution, `results_df` shows plans with HDF files already present (if any).

In [ ]:
# View initial results_df
print("\n=== Initial results_df ===")
cols = ['plan_number', 'plan_title', 'hdf_exists', 'completed', 'has_errors']
print(ras.results_df[cols].to_string())

## Execute a Plan

After execution, `results_df` is automatically updated with fresh results.

In [ ]:
# Execute plan 01
print("Executing plan 01...")
RasCmdr.compute_plan("01", num_cores=2)
print("Execution complete!")

## View Updated results_df

The `results_df` is automatically refreshed after execution.

In [ ]:
# View updated results_df
print("\n=== After Execution ===")
cols = ['plan_number', 'hdf_exists', 'completed', 'has_errors', 'has_warnings']
print(ras.results_df[cols].to_string())

## Inspect Plan 01 Details

In [ ]:
# Get plan 01 results
plan_01 = ras.results_df[ras.results_df['plan_number'] == '01'].iloc[0]

print("=== Plan 01 Summary ===")
print(f"Plan Title: {plan_01['plan_title']}")
print(f"Flow Type: {plan_01['flow_type']}")
print(f"HDF File: {Path(plan_01['hdf_path']).name}")
print(f"HDF Modified: {plan_01['hdf_file_modified']}")
print(f"RAS Version: {plan_01['ras_version']}")
print(f"\nExecution Status:")
print(f"  Completed: {plan_01['completed']}")
print(f"  Has Errors: {plan_01['has_errors']}")
print(f"  Error Count: {plan_01['error_count']}")
print(f"  Has Warnings: {plan_01['has_warnings']}")
print(f"  Warning Count: {plan_01['warning_count']}")

## Runtime Performance Data

In [ ]:
# Runtime data
print("=== Runtime Performance ===")
print(f"Simulation Start: {plan_01.get('runtime_simulation_start', 'N/A')}")
print(f"Simulation End: {plan_01.get('runtime_simulation_end', 'N/A')}")
print(f"Simulation Hours: {plan_01.get('runtime_simulation_hours', 'N/A')}")
print(f"Complete Process Hours: {plan_01.get('runtime_complete_process_hours', 'N/A')}")
print(f"Unsteady Compute Hours: {plan_01.get('runtime_unsteady_compute_hours', 'N/A')}")
print(f"Complete Process Speed: {plan_01.get('runtime_complete_process_speed', 'N/A'):.1f} hr/hr")

## Volume Accounting (Unsteady Only)

In [ ]:
# Volume accounting data
vol_cols = [c for c in ras.results_df.columns if c.startswith('vol_')]

if vol_cols and plan_01['flow_type'] == 'Unsteady':
    print("=== Volume Accounting ===")
    for col in vol_cols:
        value = plan_01.get(col, 'N/A')
        # Format numbers nicely
        if isinstance(value, (int, float)):
            print(f"{col.replace('vol_', '')}: {value:,.4f}")
        else:
            print(f"{col.replace('vol_', '')}: {value}")
else:
    print("No volume accounting data (steady flow or data not available)")

## Filter Plans by Execution Status

In [ ]:
# Find executed plans
executed = ras.results_df[ras.results_df['hdf_exists']]
print(f"\nExecuted plans: {executed['plan_number'].tolist()}")

# Find completed plans
completed = ras.results_df[ras.results_df['completed']]
print(f"Completed plans: {completed['plan_number'].tolist()}")

# Find plans with errors
with_errors = ras.results_df[ras.results_df['has_errors']]
if len(with_errors) > 0:
    print(f"Plans with errors: {with_errors['plan_number'].tolist()}")
else:
    print("No plans with errors detected")

# Find plans pending execution
pending = ras.results_df[~ras.results_df['hdf_exists']]
print(f"Plans pending execution: {pending['plan_number'].tolist()}")

## Execute Multiple Plans in Parallel

In [ ]:
# Execute remaining plans in parallel
pending_plans = ras.results_df[~ras.results_df['hdf_exists']]['plan_number'].tolist()

if pending_plans:
    print(f"\nExecuting {len(pending_plans)} plans in parallel: {pending_plans}")
    RasCmdr.compute_parallel(plan_number=pending_plans, max_workers=2, num_cores=2)
    print("Parallel execution complete!")
else:
    print("All plans already executed")

## View Complete Results Summary

In [ ]:
# Complete results summary
print("\n=== Final Results Summary ===")
summary_cols = ['plan_number', 'plan_title', 'completed', 'has_errors', 
                'runtime_complete_process_hours', 'runtime_complete_process_speed']
print(ras.results_df[summary_cols].to_string())

# Statistics
executed = ras.results_df[ras.results_df['hdf_exists']]
print(f"\n=== Summary Statistics ===")
print(f"Total plans: {len(ras.results_df)}")
print(f"Executed: {len(executed)}")
print(f"Completed successfully: {executed['completed'].sum()}")
print(f"With errors: {executed['has_errors'].sum()}")
print(f"With warnings: {executed['has_warnings'].sum()}")
print(f"Total compute time: {executed['runtime_complete_process_hours'].sum():.4f} hours")

## Manual results_df Update

You can manually refresh results_df for specific plans or all plans.

In [ ]:
# Manually update results_df for specific plans
ras.update_results_df(plan_numbers=['01'])
print("Updated results for plan 01")

# Or update all plans
ras.update_results_df()
print("Updated results for all plans")

## Using ResultsParser Directly

You can parse compute messages directly using `ResultsParser`.

In [ ]:
from ras_commander.hdf import HdfResultsPlan

# Get compute messages for a plan
plan_01_row = ras.results_df[ras.results_df['plan_number'] == '01'].iloc[0]
hdf_path = Path(plan_01_row['hdf_path'])

if hdf_path.exists():
    # Extract messages using HDF-only method (no RasControl locking)
    messages = HdfResultsPlan.get_compute_messages_hdf_only(hdf_path)
    
    print(f"Compute messages length: {len(messages)} characters")
    print("\nFirst 300 characters:")
    print(messages[:300])
    
    # Parse for errors/warnings
    parsed = ResultsParser.parse_compute_messages(messages)
    print(f"\nParsed results:")
    for key, value in parsed.items():
        print(f"  {key}: {value}")

## Benefits of results_df

1. **Quick Status Checks**: See execution status without opening HDF files
2. **Error Detection**: Automatically detects errors in compute messages
3. **Performance Metrics**: Runtime data and compute speeds available
4. **Volume Accounting**: Mass balance data for unsteady models
5. **No Locking Issues**: Uses HDF-only methods (no RasControl/COM)
6. **Automatic Updates**: Refreshes after compute_plan(), compute_parallel(), compute_test_mode()

## Cleanup

In [ ]:
# Optional: Clean up extracted project
import shutil
example_projects = project_path.parent
if example_projects.name == "example_projects":
    shutil.rmtree(example_projects, ignore_errors=True)
    print(f"Cleaned up: {example_projects}")